# Codigos

In [ ]:
import numpy as np
from scipy.stats import friedmanchisquare
from scipy.stats import rankdata
import scipy as sci
import pandas as pd

In [ ]:
# Posto para o Teste de Friedman
def postos_friedman(dados):
    return np.apply_along_axis(rankdata, axis=1, arr=dados,method='average')

# Posto para o Teste de Friedman Aligned
def postos_friedman_aligned(dados):
  # Passo 1: Calcular a média (valor de localização) de cada linha (dataset)
  location_values = np.where(np.mean(dados,axis=1)<=1e-10,0,np.mean(dados,axis=1))

  # Passo 2: Calcular as diferenças entre cada valor e o valor de localização correspondente
  aligned_observations = dados - location_values[:, np.newaxis]

  # Passo 3: Rankear todas as diferenças alinhadas
  aligned_ranks = pd.DataFrame(aligned_observations.flatten()).rank(method='average').to_numpy().reshape(dados.shape)
  return aligned_ranks

def media_postos(posto):
  mean_values = np.mean(posto, axis=0)
  return mean_values

def estatistica_FrAligned(dados):
  posto = postos_friedman_aligned(dados)
  k = np.size(posto,axis=1)
  df = k-1
  n = np.size(posto,axis=0)
  Rj = np.sum(posto,axis=0)
  Ri = np.sum(posto,axis=1)
  Ri_quadrado = Ri**2
  Rj_quadrado = Rj**2
  sumRi = np.sum(Ri_quadrado)
  sumRj = np.sum(Rj_quadrado)
  Far = ((k-1)*(sumRj - (k*n**2/4)*(k*n+1)**2))/((k*n*(k*n+1)*(2*k*n+1)/6)-((1/k)*sumRi))
  return Far,df

In [ ]:
def z_score_fri(Ri,Rj,k,n):
  z = np.abs((Ri-Rj))/(np.sqrt((k*(k+1))/(6*n)))
  return z

# IMPLEMENTAR O Z_SCORE DO FRIEDMAN ALIGNED
def z_score_frialig(Ri,Rj,k,n):
  z = np.abs((Ri - Rj))/(np.sqrt((k*(n+1))/6))
  return z

def pvalor(z):
  # Cálculo do p-valor (teste bicaudal)
  p_value = 2 * sci.stats.norm.sf(abs(z))  # sf = 1 - cdf
  return p_value

def nemenyi(p,k):
  m = k*(k-1)/2
  v = m*p
  return np.min([v,1])

# def holm(p,k,j):
#   m = k*(k-1)/2
#   if j > 0:
#     v1 = np.min([(m-j+1)*p,1])
#     v2 = np.min([(m-j)*p,1])
#     v = np.max([v1,v2])
#   else:
#     v = np.min([(m-j+1)*p,1])
#   return v

def holm(df, k):
    """Holm no DataFrame, garantindo monotonicidade."""

    m = k * (k - 1) / 2  # Número total de comparações
    p_values = df['p-valor'].values  # coluna dos p-valores
    j = len(p_values)
    # Aplicar Holm
    adjusted_p = np.minimum((m - np.arange(1, j + 1) + 1) * p_values, 1)

    # Garantir que os valores ajustados não diminuam
    for i in range(1, len(adjusted_p)):
        adjusted_p[i] = max(adjusted_p[i], adjusted_p[i - 1])

    return adjusted_p


In [ ]:
import itertools

# ARRUMAR A TABELA Z_SCORE PARA RECEBER O FRIEDMAN ALIGNED

def tabela_z_score(dados,k,n,id):
  # Gerar automaticamente todas as combinações possíveis de 2 elementos
  comparacoes = list(itertools.combinations(dados.keys(), 2))

  if id == 0:
    # Criar dicionário com soma dos valores
    comparacoes_dict = [
        (f"{key1} vs {key2}", round(z_score_fri(dados[key1][0],dados[key2][0],k,n), 4))
        for key1, key2 in itertools.combinations(dados.keys(), 2)
    ]
  elif id == 1:
    # Criar dicionário com soma dos valores
    comparacoes_dict = [
        (f"{key1} vs {key2}", round(z_score_frialig(dados[key1][0],dados[key2][0],k,n), 4))
        for key1, key2 in itertools.combinations(dados.keys(), 2)
    ]
  else:
    print('Modelo não implementado')
  # # Exibir dicionário formatado
  compar_dict = pd.DataFrame(comparacoes_dict, columns=["Comparacoes", "z-score"])

  # Ordenar do maior para o menor
  df = compar_dict.sort_values(by="z-score", ascending=False)

  # Resetar o índice (opcional, para deixar numerado de 0 até n-1)
  df = df.reset_index(drop=True)
  return df

def tabela_post_hoc(dados,k,n,id):
  df = tabela_z_score(dados,k,n,id)

  df['p-valor'] = pvalor(df['z-score'])

  df['Nemenyi'] = df["z-score"].apply(lambda p: nemenyi(pvalor(p), k))

  df['Holm'] = holm(df,k)

  df = df.round(4)

  return df

# Dados com 5 casas decimais

In [ ]:
dados1 = np.array([
    [1564.76509, 1564.76490, 1565.50791, 1564.76596, 1565.69641],
    [1.35361, 1.33997, 1.56018, 1.35996, 1.34033],
    [26.53159, 26.53133, 26.60963, 26.57612, 26.57784],
    [9.39001, 9.39001, 9.39154, 9.39043, 9.39087],
    [0.91150, 0.90509, 1.32994, 0.91450, 0.92746]
])

In [ ]:
postos_friedman(dados1)

array([[2. , 1. , 4. , 3. , 5. ],
       [3. , 1. , 5. , 4. , 2. ],
       [2. , 1. , 5. , 3. , 4. ],
       [1.5, 1.5, 5. , 3. , 4. ],
       [2. , 1. , 5. , 3. , 4. ]])

In [ ]:
media_postos(postos_friedman(dados1))

array([2.1, 1.1, 4.8, 3.2, 3.8])

In [ ]:
# Aplicando o teste de Friedman
stat, p = friedmanchisquare(*dados1.T)

print(f'Estatística de teste: {stat}')
Fcri = sci.stats.chi2.ppf(1-.05, df=4)    # valor de F critico calculado com PPF (funcao ponto de porcentagem) é inversa a CDF
print(f"Fr critico = {Fcri}")

p_valor_fr = 1-sci.stats.chi2.cdf(stat,df=4)
print(f'p_valor {p_valor_fr}')
print(f'Valor-p: {p}')

Estatística de teste: 16.848484848484855
Fr critico = 9.487729036781154
p_valor 0.0020684472508883944
Valor-p: 0.0020684472508883814


### Post-Hoc Friedman Padrao

In [ ]:
avg_rank_fri = {
"AOA" : [2.1],
"DE" : [1.1],
"PSO" : [4.8],
"SCA" : [3.2],
"GA" : [3.8]}

dados_avg_rank_fri = pd.DataFrame(avg_rank_fri)
k = 5  # numero de metodos
n = 5 # numero de problemas resolvidos

df1 = tabela_post_hoc(dados_avg_rank_fri,k,n,0)
df1

,Comparacoes,z-score,p-valor,Nemenyi,Holm
0,DE vs PSO,3.7,0.0002,0.0022,0.0022
1,AOA vs PSO,2.7,0.0069,0.0693,0.0624
2,DE vs GA,2.7,0.0069,0.0693,0.0624
3,DE vs SCA,2.1,0.0357,0.3573,0.2501
4,AOA vs GA,1.7,0.0891,0.8913,0.5348
5,PSO vs SCA,1.6,0.1096,1.0000,0.5480
6,AOA vs SCA,1.1,0.2713,1.0000,1.0000
7,AOA vs DE,1.0,0.3173,1.0000,1.0000
8,PSO vs GA,1.0,0.3173,1.0000,1.0000
9,SCA vs GA,0.6,0.5485,1.0000,1.0000


In [ ]:
# Convertendo para LaTeX sem a coluna de índices
latex_code = df1.to_latex(index=False, float_format="%.4f")

# Exibindo o código LaTeX
print(latex_code)

\begin{tabular}{lrrrr}
\toprule
Comparacoes & z-score & p-valor & Nemenyi & Holm \\
\midrule
DE vs PSO & 3.7000 & 0.0002 & 0.0022 & 0.0022 \\
AOA vs PSO & 2.7000 & 0.0069 & 0.0693 & 0.0624 \\
DE vs GA & 2.7000 & 0.0069 & 0.0693 & 0.0624 \\
DE vs SCA & 2.1000 & 0.0357 & 0.3573 & 0.2501 \\
AOA vs GA & 1.7000 & 0.0891 & 0.8913 & 0.5348 \\
PSO vs SCA & 1.6000 & 0.1096 & 1.0000 & 0.5480 \\
AOA vs SCA & 1.1000 & 0.2713 & 1.0000 & 1.0000 \\
AOA vs DE & 1.0000 & 0.3173 & 1.0000 & 1.0000 \\
PSO vs GA & 1.0000 & 0.3173 & 1.0000 & 1.0000 \\
SCA vs GA & 0.6000 & 0.5485 & 1.0000 & 1.0000 \\
\bottomrule
\end{tabular}



### Friedman Aligned - Escala Descendente

In [ ]:
postos_friedman_aligned(dados1)

array([[ 2. ,  1. , 24. ,  3. , 25. ],
       [10. ,  8. , 22. , 13. ,  9. ],
       [12. , 11. , 21. , 19. , 20. ],
       [14.5, 14.5, 18. , 16. , 17. ],
       [ 5. ,  4. , 23. ,  6. ,  7. ]])

In [ ]:
media_postos(postos_friedman_aligned(dados1))

array([ 8.7,  7.7, 21.6, 11.4, 15.6])

In [ ]:
Fra,df = estatistica_FrAligned(dados1)
print(f'Fra = {Fra}')
print(f'df = {df}')

Fra = 11.931275266446164
df = 4


In [ ]:
p_valor_fra = 1-sci.stats.chi2.cdf(Fra,df=df)
p_valor_fra

0.017869704572575862

In [ ]:
# valor de F critico calculado com PPF (funcao ponto de porcentagem) é inversa a CDF
Fcri_1perc = sci.stats.chi2.ppf(1-.01, df=df)
Fcri_5perc = sci.stats.chi2.ppf(1-.05, df=df)
Fcri_10perc = sci.stats.chi2.ppf(1-.1, df=df)

print(f'Valor critico 01%: = {Fcri_1perc}')
print(f'Valor critico 05%: = {Fcri_5perc}')
print(f'Valor critico 10%: = {Fcri_10perc}')

Valor critico 01%: = 13.276704135987622
Valor critico 05%: = 9.487729036781154
Valor critico 10%: = 7.779440339734858


In [ ]:
avg_rank_fri_alig = {
"AOA" : [8.7],
"DE" : [7.7],
"PSO" : [21.6],
"SCA" : [11.4],
"GA" : [15.6]}

dados_avg_rank_fri_alig = pd.DataFrame(avg_rank_fri_alig)
k = 5  # numero de metodos
n = 5 # numero de problemas resolvidos

df2 = tabela_post_hoc(dados_avg_rank_fri_alig,k,n,1)
df2

,Comparacoes,z-score,p-valor,Nemenyi,Holm
0,DE vs PSO,6.2163,0.0000,0.0000,0.0000
1,AOA vs PSO,5.7691,0.0000,0.0000,0.0000
2,PSO vs SCA,4.5616,0.0000,0.0001,0.0000
3,DE vs GA,3.5330,0.0004,0.0041,0.0029
4,AOA vs GA,3.0858,0.0020,0.0203,0.0122
5,PSO vs GA,2.6833,0.0073,0.0729,0.0364
6,SCA vs GA,1.8783,0.0603,0.6034,0.2414
7,DE vs SCA,1.6547,0.0980,0.9799,0.2940
8,AOA vs SCA,1.2075,0.2272,1.0000,0.4545
9,AOA vs DE,0.4472,0.6547,1.0000,0.6547


In [ ]:
# Convertendo para LaTeX sem a coluna de índices
latex_code2 = df2.to_latex(index=False, float_format="%.4f")

# Exibindo o código LaTeX
print(latex_code2)

\begin{tabular}{lrrrr}
\toprule
Comparacoes & z-score & p-valor & Nemenyi & Holm \\
\midrule
DE vs PSO & 6.2163 & 0.0000 & 0.0000 & 0.0000 \\
AOA vs PSO & 5.7691 & 0.0000 & 0.0000 & 0.0000 \\
PSO vs SCA & 4.5616 & 0.0000 & 0.0001 & 0.0000 \\
DE vs GA & 3.5330 & 0.0004 & 0.0041 & 0.0029 \\
AOA vs GA & 3.0858 & 0.0020 & 0.0203 & 0.0122 \\
PSO vs GA & 2.6833 & 0.0073 & 0.0729 & 0.0364 \\
SCA vs GA & 1.8783 & 0.0603 & 0.6034 & 0.2414 \\
DE vs SCA & 1.6547 & 0.0980 & 0.9799 & 0.2940 \\
AOA vs SCA & 1.2075 & 0.2272 & 1.0000 & 0.4545 \\
AOA vs DE & 0.4472 & 0.6547 & 1.0000 & 0.6547 \\
\bottomrule
\end{tabular}



# Dados com 2 casas decimais

In [ ]:
dados2 = np.array([
    [1564.77, 1564.76, 1565.51, 1564.77, 1565.70],
    [1.35, 1.34, 1.56, 1.36, 1.34],
    [26.53, 26.53, 26.61, 26.58, 26.58],
    [9.39, 9.39, 9.39, 9.39, 9.39],
    [0.91, 0.91, 1.33, 0.91, 0.93]
])

In [ ]:
# Aplicando o teste de Friedman
stat2, p2 = friedmanchisquare(*dados2.T)

print(f'Estatística de teste: {stat}')
p2_valor_fr = 1-sci.stats.chi2.cdf(stat2,df=4)
print(f'p_valor {p2_valor_fr}')
print(f'Valor-p: {p2}')

Estatística de teste: 16.848484848484855
p_valor 0.027841543277494996
Valor-p: 0.027841543277495023


# Dadps GArcia

In [ ]:
dado_garcia= np.array([
    [-0.024,-0.003, 0.009, 0.019],
    [-0.001, 0.020,-0.004,-0.015],
    [0.068, 0.047,-0.084,-0.031],
    [0.112,-0.020,-0.002,-0.091],
    [-0.030,0.016, 0.037,-0.023],
    [ 0.022 , 0.022 ,-0.001,-0.043],
    [ 0.136 , 0.040, 0.123 ,-0.299],
    [0.025 , 0.001 , 0.013 ,-0.038],
    [ 0.027 ,-0.016 , 0.001 ,-0.013],
    [ 0.069 ,-0.068 , 0.030 ,-0.032],
    [-0.005 , 0.002 ,-0.002 , 0.005],
    [0.010 ,-0.010 ,-0.003 , 0.003],
    [0.063,-0.017, 0.032,-0.078],
    [0.152 ,0.146,-0.363 , 0.065],
    [0.012 , 0.012 , 0.002 ,-0.026],
    [0.108 , 0.078, 0.058 ,-0.244],
    [0.120,-0.085 ,-0.025 ,-0.010],
    [0.038 ,-0.028 , 0.026 ,-0.036],
    [-0.008 , 0.012 , 0.022 ,-0.026],
    [0.055, 0.018 ,0.008 ,-0.065 ],
    [-0.001, 0.011 , 0.000 ,-0.010],
    [0.178,-0.016 ,-0.057 ,-0.105],
    [0.024 , 0.007 ,-0.004 ,-0.027],
    [-0.003,-0.002 , 0.003 , 0.003]
])

In [ ]:
dado_garcia

array([[-0.024, -0.003,  0.009,  0.019],
       [-0.001,  0.02 , -0.004, -0.015],
       [ 0.068,  0.047, -0.084, -0.031],
       [ 0.112, -0.02 , -0.002, -0.091],
       [-0.03 ,  0.016,  0.037, -0.023],
       [ 0.022,  0.022, -0.001, -0.043],
       [ 0.136,  0.04 ,  0.123, -0.299],
       [ 0.025,  0.001,  0.013, -0.038],
       [ 0.027, -0.016,  0.001, -0.013],
       [ 0.069, -0.068,  0.03 , -0.032],
       [-0.005,  0.002, -0.002,  0.005],
       [ 0.01 , -0.01 , -0.003,  0.003],
       [ 0.063, -0.017,  0.032, -0.078],
       [ 0.152,  0.146, -0.363,  0.065],
       [ 0.012,  0.012,  0.002, -0.026],
       [ 0.108,  0.078,  0.058, -0.244],
       [ 0.12 , -0.085, -0.025, -0.01 ],
       [ 0.038, -0.028,  0.026, -0.036],
       [-0.008,  0.012,  0.022, -0.026],
       [ 0.055,  0.018,  0.008, -0.065],
       [-0.001,  0.011,  0.   , -0.01 ],
       [ 0.178, -0.016, -0.057, -0.105],
       [ 0.024,  0.007, -0.004, -0.027],
       [-0.003, -0.002,  0.003,  0.003]])

In [ ]:
media_da_linha = np.where(np.mean(dado_garcia,axis=1)<=1e-10,0,np.mean(dado_garcia,axis=1))
media_da_linha

array([0.00025, 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ,
       0.00025, 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ,
       0.     , 0.     , 0.     , 0.     , 0.     , 0.004  , 0.     ,
       0.     , 0.     , 0.00025])

In [ ]:
aligned_observations = dado_garcia - media_da_linha[:, np.newaxis]
print(f"{aligned_observations}\n")

[[-0.02425 -0.00325  0.00875  0.01875]
 [-0.001    0.02    -0.004   -0.015  ]
 [ 0.068    0.047   -0.084   -0.031  ]
 [ 0.112   -0.02    -0.002   -0.091  ]
 [-0.03     0.016    0.037   -0.023  ]
 [ 0.022    0.022   -0.001   -0.043  ]
 [ 0.136    0.04     0.123   -0.299  ]
 [ 0.02475  0.00075  0.01275 -0.03825]
 [ 0.027   -0.016    0.001   -0.013  ]
 [ 0.069   -0.068    0.03    -0.032  ]
 [-0.005    0.002   -0.002    0.005  ]
 [ 0.01    -0.01    -0.003    0.003  ]
 [ 0.063   -0.017    0.032   -0.078  ]
 [ 0.152    0.146   -0.363    0.065  ]
 [ 0.012    0.012    0.002   -0.026  ]
 [ 0.108    0.078    0.058   -0.244  ]
 [ 0.12    -0.085   -0.025   -0.01   ]
 [ 0.038   -0.028    0.026   -0.036  ]
 [-0.008    0.012    0.022   -0.026  ]
 [ 0.051    0.014    0.004   -0.069  ]
 [-0.001    0.011    0.      -0.01   ]
 [ 0.178   -0.016   -0.057   -0.105  ]
 [ 0.024    0.007   -0.004   -0.027  ]
 [-0.00325 -0.00225  0.00275  0.00275]]



In [ ]:
bb = pd.DataFrame(aligned_observations.flatten()).rank(ascending=False).to_numpy().reshape(dado_garcia.shape)
bb

array([[74. , 58. , 39. , 30. ],
       [52. , 29. , 60.5, 68. ],
       [11. , 16. , 90. , 81. ],
       [ 7. , 72. , 54.5, 92. ],
       [80. , 31. , 19. , 73. ],
       [27. , 27. , 52. , 85. ],
       [ 4. , 17. ,  5. , 95. ],
       [24. , 49. , 33. , 84. ],
       [22. , 69.5, 48. , 67. ],
       [10. , 87. , 21. , 82. ],
       [62. , 46.5, 54.5, 41. ],
       [38. , 65. , 57. , 43. ],
       [13. , 71. , 20. , 89. ],
       [ 2. ,  3. , 96. , 12. ],
       [35. , 35. , 46.5, 76.5],
       [ 8. ,  9. , 14. , 94. ],
       [ 6. , 91. , 75. , 65. ],
       [18. , 79. , 23. , 83. ],
       [63. , 35. , 27. , 76.5],
       [15. , 32. , 42. , 88. ],
       [52. , 37. , 50. , 65. ],
       [ 1. , 69.5, 86. , 93. ],
       [25. , 40. , 60.5, 78. ],
       [59. , 56. , 44.5, 44.5]])

In [ ]:
np.sum(bb,axis=0)

array([ 708. , 1124.5, 1118. , 1705.5])

In [ ]:
np.mean(bb,axis=0)

array([29.5       , 46.85416667, 46.58333333, 71.0625    ])

In [ ]:
estatistica_FrAligned(dado_garcia)

(22.370433283051057, 3)

In [ ]:
posto = postos_friedman_aligned(dado_garcia)
k = np.size(posto,axis=1)
n = np.size(posto,axis=0)
Rj = np.sum(posto,axis=0)
Ri = np.sum(posto,axis=1)
print(k)
print(n)
print(Rj)
print(Ri)

4
24
[1620.  1203.5 1210.   622.5]
[187.  178.5 190.  162.5 185.  197.  267.  198.  181.5 188.  184.  185.
 195.  275.  195.  263.  151.  185.  186.5 211.  184.  138.5 184.5 184. ]


In [ ]:
def postos_friedman_aligned(dados):
  # Passo 1: Calcular a média (valor de localização) de cada linha (dataset)
  location_values = np.mean(dados, axis=1)

  # Passo 2: Calcular as diferenças entre cada valor e o valor de localização correspondente
  aligned_observations = dados - location_values[:, np.newaxis]

  # Passo 3: Rankear todas as diferenças alinhadas
  aligned_ranks = pd.DataFrame(aligned_observations.flatten()).rank(ascending=False,method='first').to_numpy().reshape(dados.shape)
  return aligned_ranks


# Post - Hoc no Golcuk

In [ ]:
ar_golcuk = {
"EDP" : [6.2895],
"AP" : [5.5000],
"SP" : [5.1842],
"DP" : [5.0526],
"EDPSF" :[4.7632],
"DPSF" :[ 4.1579],
"EP" : [2.7368],
"ITCH" : [2.3158]}

df = pd.DataFrame(ar_golcuk)
print(ar_golcuk)

{'EDP': [6.2895], 'AP': [5.5], 'SP': [5.1842], 'DP': [5.0526], 'EDPSF': [4.7632], 'DPSF': [4.1579], 'EP': [2.7368], 'ITCH': [2.3158]}


In [ ]:
import itertools

k = 8  # numero de metodos
n = 19 # numero de problemas resolvidos

# Gerar automaticamente todas as combinações possíveis de 2 elementos
comparacoes = list(itertools.combinations(ar_golcuk.keys(), 2))

# Criar dicionário com soma dos valores
comparacoes_dict = [
    (f"{key1} vs {key2}", round(z_score_fri(ar_golcuk[key1][0],ar_golcuk[key2][0],k,n), 4))
    for key1, key2 in itertools.combinations(ar_golcuk.keys(), 2)
]

# # Exibir dicionário formatado
compar_dict = pd.DataFrame(comparacoes_dict, columns=["Confronto", "Valor"])

# Ordenar do maior para o menor
df = compar_dict.sort_values(by="Valor", ascending=False)

# Resetar o índice (opcional, para deixar numerado de 0 até n-1)
df = df.reset_index(drop=True)

df['p-valor'] = pvalor(df['Valor'])
# Exibir a tabela ordenada

df['Nemenyi'] = df["Valor"].apply(lambda p: nemenyi(pvalor(p), k))

#df['Holm'] = df.apply(lambda p: holm(pvalor(p['Valor']),k,(p.name+1)), axis=1).round(4)

df['Holm'] = holm(df,k)

df = df.round(3)
df



,Confronto,Valor,p-valor,Nemenyi,Holm
0,EDP vs ITCH,5.000,0.000,0.000,0.000
1,EDP vs EP,4.470,0.000,0.000,0.000
2,AP vs ITCH,4.007,0.000,0.002,0.002
3,SP vs ITCH,3.609,0.000,0.009,0.008
4,AP vs EP,3.477,0.001,0.014,0.012
5,DP vs ITCH,3.444,0.001,0.016,0.013
6,SP vs EP,3.080,0.002,0.058,0.046
7,EDPSF vs ITCH,3.080,0.002,0.058,0.046
8,DP vs EP,2.914,0.004,0.100,0.071
9,EDP vs DPSF,2.682,0.007,0.205,0.139


In [ ]:
ar_golcuk = {
"EDP" : [6.2895],
"AP" : [5.5000],
"SP" : [5.1842],
"DP" : [5.0526],
"EDPSF" :[4.7632],
"DPSF" :[ 4.1579],
"EP" : [2.7368],
"ITCH" : [2.3158]}

dados_golcuk = pd.DataFrame(ar_golcuk)
k = 8  # numero de metodos
n = 19 # numero de problemas resolvidos

tabela_post_hoc(dados_golcuk,k,n)

,Comparacoes,z-score,p-valor,Nemenyi,Holm
0,EDP vs ITCH,5.000,0.000,0.000,0.000
1,EDP vs EP,4.470,0.000,0.000,0.000
2,AP vs ITCH,4.007,0.000,0.002,0.002
3,SP vs ITCH,3.609,0.000,0.009,0.008
4,AP vs EP,3.477,0.001,0.014,0.012
5,DP vs ITCH,3.444,0.001,0.016,0.013
6,SP vs EP,3.080,0.002,0.058,0.046
7,EDPSF vs ITCH,3.080,0.002,0.058,0.046
8,DP vs EP,2.914,0.004,0.100,0.071
9,EDP vs DPSF,2.682,0.007,0.205,0.139


In [ ]:
p = pvalor(3.477)
k = 8
j = 5
holm(p,k,j)

0.012169387623800933

In [ ]:
round(p,3)

0.001

In [ ]:
j = 5
np.arange(1, j + 1)


array([1, 2, 3, 4, 5])